In [1]:
import sys
import os

# preamble for jupyter notebook and django
import django
import platform

if platform.node() == "srv-mcc-apsis":
    sys.path.append("/home/galm/software/django/tmv/BasicBrowser/")
    #sys.path.append('/home/leey/tmv/BasicBrowser/')
else:
    # local paths
    sys.path.append('/home/leey/Documents/Data/tmv/BasicBrowser/')

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

# import from appended path
import parliament.models as pm
from parliament.tasks import do_search, run_tm
import cities.models as cmodels
from django.contrib.auth.models import User
from tmv_app.models import *
from utils.tm_mgmt import update_topic_scores
from django.db.models import Q, Count, Func, F, Sum, Avg, Value as V
from django.core import serializers

/home/leey/venv/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)
Slow version of gensim.models.doc2vec is being used
Slow version of Fasttext is being used


In [2]:
from mdb_stammdata_parser import *

In [9]:
#pm.Person.objects.filter(information_source="MDB Stammdata").all().delete()

(14405,
 {'parliament.Interjection_persons': 0,
  'parliament.ListMembership': 0,
  'parliament.Seat': 7800,
  'parliament.ConstituencyVote1': 0,
  'parliament.Post': 0,
  'parliament.Person': 6605})

In [8]:
#pm.Utterance.objects.filter(speaker__active_country=cmodels.Country.objects.get(name="Germany").id).delete()

(8994,
 {'scoping.Note': 0,
  'scoping.DocOwnership': 0,
  'tmv_app.DocTopic': 0,
  'parliament.Utterance_search_matches': 0,
  'parliament.Paragraph_search_matches': 0,
  'parliament.Interjection_parties': 2626,
  'parliament.Interjection_persons': 916,
  'parliament.Interjection': 2586,
  'parliament.Paragraph': 2423,
  'parliament.Utterance': 443})

In [3]:
parse_mdb_data()

read data from ./mdbs/MDB_STAMMDATEN.XML
<Element 'DOCUMENT' at 0x7f8182c38868>
Done. 24 warnings.
